# Analyze 

In [1]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import datetime
import json
import folium
import utils
from utils import categorize_time_to_half_hour, categorize_time_to_half_hour_numerical, dt_to_time, time_to_time
import consts

## Defines

In [2]:
USE_GRID = 'hex'

In [3]:
if USE_GRID == 'nuts':
    path_timedata = consts.PATH_TIMEDATA_NUTS
    path_grid = consts.PATH_MASTERNUTS
    path_map = consts.PATH_DOCS / 'sunrise-nuts.html'
elif USE_GRID == 'hex':
    path_timedata = consts.PATH_TIMEDATA_HEX
    path_grid = consts.PATH_HEXAGON
    path_map = consts.PATH_DOCS / 'sunrise-hex.html'
else:
    raise ValueError(f"Unknown grid type {USE_GRID}")

## Load data

In [4]:
# Load timedata
time_data = json.load(open(path_timedata, 'r', encoding='utf-8'))

# Load Nuts3
gdf_grid_raw = gpd.read_file(path_grid)

# Reduce precision
gdf_grid_raw['geometry'] = gdf_grid_raw['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))

# Load Countryborders
gdf_borders = gpd.read_file(consts.PATH_BORDERS)
gdf_borders['geometry'] = gdf_borders['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))

## Prepare Dataset

In [5]:
# Serialize JSON
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],
    'name': x['name'],
    'sunrise_current': x['sunrise']['current']['latest'],
    'sunrise_summertime': x['sunrise']['summer']['latest'],
    'sunrise_wintertime': x['sunrise']['winter']['latest'],
    # 'sunrise_current': x['sunrise']['current']['earliest'],
    # 'sunrise_summertime': x['sunrise']['summer']['earliest'],
    # 'sunrise_wintertime': x['sunrise']['winter']['earliest'],
}, time_data))

df_timedata = pd.DataFrame(df_timedata)

# Convert strings to time
df_timedata['sunrise_current'] = df_timedata['sunrise_current'].apply(time_to_time)
df_timedata['sunrise_summertime'] = df_timedata['sunrise_summertime'].apply(time_to_time)
df_timedata['sunrise_wintertime'] = df_timedata['sunrise_wintertime'].apply(time_to_time)

# Current System
df_timedata['sunrise_current_category'] = df_timedata['sunrise_current'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_current_numerical'] = df_timedata['sunrise_current'].apply(
    categorize_time_to_half_hour_numerical
)

# Everything on Summertime
df_timedata['sunrise_summertime_category'] = df_timedata['sunrise_summertime'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_summertime_numerical'] = df_timedata['sunrise_summertime'].apply(
    categorize_time_to_half_hour_numerical
)

# Everything on Wintertime
df_timedata['sunrise_wintertime_category'] = df_timedata['sunrise_wintertime'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_wintertime_numerical'] = df_timedata['sunrise_wintertime'].apply(
    categorize_time_to_half_hour_numerical
)

# Merge Dataframes
gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')

In [20]:
m = folium.Map(
    tiles=None,
    prefer_canvas=False
)

folium.TileLayer(
    'cartodbpositron',
    opacity=0.4
).add_to(m)

# Create bins evenly distributed with 0 in the center
min_value = min(
    gdf_grid['sunrise_current_numerical'].min(),
    gdf_grid['sunrise_summertime_numerical'].min(),
    gdf_grid['sunrise_wintertime_numerical'].min()
)
max_value = max(
    gdf_grid['sunrise_current_numerical'].max(),
    gdf_grid['sunrise_summertime_numerical'].max(),
    gdf_grid['sunrise_wintertime_numerical'].max()
)
bins = [x * 0.5 for x in range(int(min_value * 2), int(max_value * 2) + 1)]

def add_choropleth(name, field, popup_fields):

    cp = folium.Choropleth(
        geo_data=gdf_grid.to_json(default=str),
        name=name,
        data=df_timedata,
        columns=["nuts_id", field],
        key_on="feature.properties.NUTS_ID",
        fill_color="Greys",
        fill_opacity=1,
        line_opacity=0.4,
        bins = bins,
        overlay=True,
        show=(name == "Aktuelles System")
    ).add_to(m)


    folium.GeoJsonTooltip(["name"] + popup_fields).add_to(cp.geojson)    

add_choropleth("Aktuelles System", 'sunrise_current_numerical', ['sunrise_current_category', 'sunrise_current'])
add_choropleth("Sommerzeit", 'sunrise_summertime_numerical', ['sunrise_summertime_category', 'sunrise_summertime'])
add_choropleth("Winterzeit", 'sunrise_wintertime_numerical', ['sunrise_wintertime_category', 'sunrise_wintertime'])

# Add Country borders
boudaries = folium.GeoJson(
    gdf_borders,
    name="Ländergrenzen",
    style_function=lambda x: {
        'color': 'white',
        'weight': 1,
        'fillOpacity': 0,
        'interactive':False
    },
    control=False
).add_to(m)
m.keep_in_front(boudaries)

m.fit_bounds([[34.615, -10.334], [71.114, 34.5896]])

folium.LayerControl(collapsed=False).add_to(m)

utils.addTitle(m, "Spätester Sonnenaufgang")

m.save(path_map)

## Create Hourly-Map

In [17]:
# Load timedata
time_data_hourly = json.load(open(consts.PATH_TIMEDATA_HOURLY_HEX, 'r', encoding='utf-8'))

In [19]:
# Create style dict
styledict = {}

colors = {
    'night': 'black',
    'dawn': '#312a7d',
    'sunrise': '#b48ad1',
    'day': '#e1cde1',
}

for cell in time_data_hourly:

    styledict[cell['nuts_id']] = {}
    for step in cell['sunrise']['current']['latest']:

        key = '2025-01-01T{time}'.format(
            time=step['time']
        )
        
        key = int(datetime.datetime.strptime(key, '%Y-%m-%dT%H:%M').timestamp())

        styledict[cell['nuts_id']][key] = {
            'color': colors[step['elevation_category']],
            'opacity': 1,
            'cat': step['elevation_category'],
            'elevation': step['elevation'],
            'time': step['time']
        }

        # e = step['elevation']
        # if e < -6:
        #     color = 'black'
        # elif e < 0: #
        #     color = "#5238a7"
        # elif e < 6:
        #     color = "#764bc5"
        # elif e < 12:
        #     color = "#976acd"
        # elif e < 18:
        #     color = "#b48ad1"
        # else:
        #     color = "#e1cde1"


        # styledict[cell['nuts_id']][key] = {
        #     'color': color,
        #     'opacity': 1
        # }


In [20]:
import folium
from folium.plugins import TimeSliderChoropleth

m = folium.Map()

gdf = gdf_grid.set_index('NUTS_ID')

TimeSliderChoropleth(
    gdf.to_json(default=str),
    styledict=styledict,
    init_timestamp=0,
    stroke_width=0,
    date_options='HH:mm'

).add_to(m)


m.fit_bounds([[34.615, -10.334], [71.114, 34.5896]])

m.save(consts.PATH_DOCS / 'sunrise-hex-hourly.html')